# Getting up to speed with Dask

## Part 3: Scale up!

We will do the same analysis as Part 1 & 2 but now with a Dask cluster!

AWS EC2 instance types
- (notebook): r5.xlarge (2 CPU, 16GB RAM)
- (10 workers): r5.2xlarge (8 CPU, 64GB RAM)


We are running in [Saturn Cloud](https://www.saturncloud.io/) so we are using a `SaturnCluster`, but Dask supports many other cluster deployment tools such as [YARN](https://yarn.dask.org/en/latest/) or [Kubernetes](https://docs.dask.org/en/latest/setup/kubernetes.html)

In [1]:
from dask.distributed import Client
from dask_saturn import SaturnCluster

cluster = SaturnCluster()
client = Client(cluster)
cluster

[2020-07-15 03:33:04] INFO - dask-saturn | Cluster is ready


In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import datetime
import s3fs

seed = 42

# Load and explore data

The worker nodes are different machines, so they do not have the same `data` folder as the Jupyter server. This is good, because it would be expensive to shuttle the same data to all the nodes! Because of this, we will pull directly from S3.

In [3]:
taxi_dtypes = {
    'store_and_fwd_flag': str,
    'RatecodeID': 'float64',
    'VendorID': 'float64',
    'passenger_count': 'float64',
    'payment_type': 'float64',
}

In [4]:
%%time

taxi = dd.read_csv(
    's3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv',
    dtype=taxi_dtypes, 
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
    storage_options={'anon': True},
)

CPU times: user 113 ms, sys: 35.4 ms, total: 149 ms
Wall time: 1.46 s


In [5]:
%%time
len(taxi)

CPU times: user 214 ms, sys: 4.51 ms, total: 218 ms
Wall time: 18.1 s


84399019

In [6]:
%%time
taxi.memory_usage(deep=True).sum().compute() / 1e9

CPU times: user 299 ms, sys: 12 ms, total: 311 ms
Wall time: 21.8 s


17.04023366

In [7]:
%%time
np.round(taxi.describe().compute(), 3).T

CPU times: user 3.79 s, sys: 75.8 ms, total: 3.86 s
Wall time: 24.4 s


,count,mean,std,min,25%,50%,75%,max
VendorID,84152418.0,1.645,0.498,1.00,1.00,2.00,2.00,4.00
passenger_count,84152418.0,1.563,1.208,0.00,1.00,1.00,2.00,9.00
trip_distance,84399019.0,3.001,8.091,-37264.53,1.07,1.93,8.82,45977.22
RatecodeID,84152418.0,1.061,0.760,1.00,1.00,1.00,1.00,99.00
PULocationID,84399019.0,163.158,66.016,1.00,132.00,162.00,234.00,265.00
DOLocationID,84399019.0,161.353,70.251,1.00,116.00,163.00,236.00,265.00
payment_type,84152418.0,1.289,0.479,1.00,1.00,1.00,2.00,5.00
fare_amount,84399019.0,13.344,174.375,-1856.00,7.00,11.00,32.04,943274.80
extra,84399019.0,1.087,1.249,-60.00,0.00,1.00,3.00,535.38
mta_tax,84399019.0,0.495,0.067,-0.50,0.50,0.50,0.50,212.42


# Feature engineering

In [8]:
def make_features(df):
    """ Same code from Part 1 """
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.weekofyear
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df['pickup_year_seconds'] = (df.tpep_pickup_datetime - datetime.datetime(2019, 1, 1, 0, 0, 0)).dt.seconds
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['store_and_fwd_flag'] = (df.store_and_fwd_flag == 'Y').astype(int)
    df['VendorID'] = df.VendorID.fillna(-1)
    df['RatecodeID'] = df.RatecodeID.fillna(-1)

In [9]:
%%time

make_features(taxi)

CPU times: user 55.7 ms, sys: 150 µs, total: 55.9 ms
Wall time: 53.4 ms


In [10]:
%%time

taxi.head()

CPU times: user 19.6 ms, sys: 148 µs, total: 19.8 ms
Wall time: 6.36 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_minute,pickup_year_seconds,pickup_week_hour
0,1.0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,1.0,0,151,239,1.0,...,0.0,0.3,9.95,NaN,1,1,0,46,2800,24
1,1.0,2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.6,1.0,0,239,246,1.0,...,0.0,0.3,16.30,NaN,1,1,0,59,3587,24
2,2.0,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.0,1.0,0,236,236,1.0,...,0.0,0.3,5.80,NaN,4,51,13,48,49710,109
3,2.0,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.0,1.0,0,193,193,2.0,...,0.0,0.3,7.55,NaN,2,48,15,52,57145,63
4,2.0,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.0,2.0,0,193,193,2.0,...,0.0,0.3,55.55,NaN,2,48,15,56,57417,63


<br>

If you have the RAM, you can call `df.persist()` to avoid repeated CSV loading. This returns a [future](https://docs.dask.org/en/latest/futures.html) which continues to execute in the background until it's complete.

In [11]:
taxi = taxi.persist()

In [12]:
%%time
len(taxi)

CPU times: user 592 ms, sys: 11.8 ms, total: 604 ms
Wall time: 20.8 s


84399019

In [13]:
%%time
np.round(taxi.describe().compute(), 3).T

CPU times: user 5.15 s, sys: 71.5 ms, total: 5.22 s
Wall time: 12 s


,count,mean,std,min,25%,50%,75%,max
VendorID,84399019.0,1.638,0.517,-1.00,1.00,2.00,2.00,4.00
passenger_count,84152418.0,1.563,1.208,0.00,1.00,1.00,2.00,9.00
trip_distance,84399019.0,3.001,8.091,-37264.53,1.07,1.93,8.82,45977.22
RatecodeID,84399019.0,1.055,0.767,-1.00,1.00,1.00,1.00,99.00
store_and_fwd_flag,84399019.0,0.008,0.090,0.00,0.00,0.00,0.00,1.00
PULocationID,84399019.0,163.158,66.016,1.00,132.00,162.00,234.00,265.00
DOLocationID,84399019.0,161.353,70.251,1.00,116.00,163.00,236.00,265.00
payment_type,84152418.0,1.289,0.479,1.00,1.00,1.00,2.00,5.00
fare_amount,84399019.0,13.344,174.375,-1856.00,7.00,11.00,32.04,943274.80
extra,84399019.0,1.087,1.249,-60.00,0.00,1.00,3.00,535.38


# Machine learning

## Hyperparameter search

In many cases, you don't really need all the data to train a model. Sampling is useful because if the dataset fits in RAM, many more algorithms built from `scikit-learn` can be used, rather than a few algorithms that have been re-written for parallel training.

This is where a [large hyperparameter search + Dask](https://ml.dask.org/hyper-parameter-search.html) comes in!

Let's start with a sample from the full year of data

In [14]:
# very small sample for illustration purposes, will work with larger
taxi_sample = taxi.sample(frac=0.01, replace=False, random_state=seed)

In [15]:
taxi_sample.tpep_pickup_datetime.dt.month.value_counts().compute()

3     78321
1     76684
5     75663
4     74330
10    72135
2     70188
6     69403
12    68964
11    68780
9     65680
7     63104
8     60735
Name: tpep_pickup_datetime, dtype: int64

In [16]:
len(taxi_sample), taxi_sample.memory_usage(deep=True).sum().compute() / 1e9

(843987, 0.1687974)

In [17]:
taxi_sample = taxi_sample.persist()

In [18]:
# same as Part 1
numeric_feat = [
    'pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_minute', 
    'pickup_year_seconds',
    'pickup_week_hour', 
    'passenger_count',
]
categorical_feat = [
    'VendorID', 
    'RatecodeID', 
    'store_and_fwd_flag',
    'PULocationID',
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'total_amount'

Notice that we still use `dask_ml` classes for the preprocessing, but the model is from `sklearn`.

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet

from dask_ml.compose import ColumnTransformer
from dask_ml.impute import SimpleImputer
from dask_ml.preprocessing import StandardScaler, DummyEncoder, Categorizer
from dask_ml.model_selection import GridSearchCV

pipeline = Pipeline(steps=[
    ('categorize', Categorizer(columns=categorical_feat)),
    ('onehot', DummyEncoder(columns=categorical_feat)),
    ('scale', ColumnTransformer(transformers=[('num', StandardScaler(), numeric_feat)])),
    ('impute', SimpleImputer()),
    ('clf', ElasticNet(normalize=False, max_iter=100)),
])

# 400 configurations
params = {
    'clf__l1_ratio': np.arange(0, 1.01, 0.01),
    'clf__alpha': [0, 0.5, 1, 2],
}

grid_search = GridSearchCV(pipeline, params, cv=3, scoring='neg_mean_squared_error')

In [20]:
%%time
_ = grid_search.fit(taxi_sample[features], taxi_sample[y_col])
grid_search.best_score_

CPU times: user 2.03 s, sys: 211 ms, total: 2.25 s
Wall time: 1min 27s


-224.77377848194894

## Homework

Try pulling the sample into pandas dataframes and running the grid search with vanilla `scikit-learn`, you'll really notice the difference in execution time!